# Autism Audio Pipeline Snapshot

This notebook captures the standalone scripts currently used for the autism audio pipeline (feature extraction, classical model training, and Streamlit UI). It mirrors the working production code so the project remains self-contained within `ASD_ADHD_Detection/`.


## 1. MFCC Feature Extraction (`mfcc_extract.py`)

This script scans the recordings directory, extracts 40 MFCC coefficients per frame, and saves each clip's feature matrix to the shared `features/` folder.


In [ ]:
import os
from pathlib import Path

import librosa
import numpy as np

# ---------------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------------

ROOT_DIR = Path(__file__).resolve().parents[2]  # points to /Users/.../AIML
RECORDINGS_DIR_CANDIDATES = [
    ROOT_DIR / "recordings",  # legacy location
    ROOT_DIR / "ASD_ADHD_Detection" / "recordings",  # current storage
]
FEATURES_DIR = ROOT_DIR / "features"
SR = 22050
N_MFCC = 40


def extract_mfcc_features(audio_path: Path) -> np.ndarray:
    """Load audio file and compute MFCC feature matrix."""
    y, _ = librosa.load(audio_path, sr=SR, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC)
    return mfcc


def run_mfcc_pipeline() -> None:
    recordings_dir = next((p for p in RECORDINGS_DIR_CANDIDATES if p.exists()), None)
    if recordings_dir is None:
        raise FileNotFoundError(
            "Recordings directory not found. Checked: "
            + ", ".join(str(p) for p in RECORDINGS_DIR_CANDIDATES)
        )

    FEATURES_DIR.mkdir(parents=True, exist_ok=True)

    audio_files = sorted(
        p
        for p in recordings_dir.iterdir()
        if p.suffix.lower() in {".m4a", ".wav", ".mp3"}
    )
    if not audio_files:
        print("No audio files found in recordings directory.")
        return

    print(f"Extracting MFCC features for {len(audio_files)} audio files...")
    for audio_path in audio_files:
        try:
            mfcc = extract_mfcc_features(audio_path)
            feature_name = audio_path.name + ".npy"
            np.save(FEATURES_DIR / feature_name, mfcc)
            print(f"  ✓ {audio_path.name} -> {feature_name} ({mfcc.shape})")
        except Exception as exc:  # noqa: BLE001
            print(f"  ✗ Failed to process {audio_path.name}: {exc}")


if __name__ == "__main__":
    run_mfcc_pipeline()



## 2. Classical Model Training (`model.py`)

Trains Random Forest, SVM, Naive Bayes, and an MLP on the averaged MFCC features stored in `features/`, then persists the resulting models as `.pkl` files.


In [ ]:
import os
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib

ROOT_DIR = Path(__file__).resolve().parents[2]
FEATURES_DIR = ROOT_DIR / "features"
MODEL_FILES = {
    "rf.pkl": RandomForestClassifier(n_estimators=100, random_state=42),
    "svm.pkl": SVC(kernel="rbf", C=1, gamma="scale", random_state=42),
    "nb.pkl": GaussianNB(),
    "ann.pkl": MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
}


def load_and_average_data(file_patterns):
    data_list = []
    for file_name in sorted(file_patterns):
        mfcc = np.load(FEATURES_DIR / file_name)
        data_list.append(np.mean(mfcc, axis=1))
    return np.vstack(data_list)


def build_training_set():
    autistic_files = [f for f in os.listdir(FEATURES_DIR) if f.startswith("aut_")]
    non_autistic_files = [f for f in os.listdir(FEATURES_DIR) if f.startswith("split-")]
    autistic_data = load_and_average_data(autistic_files)
    non_autistic_data = load_and_average_data(non_autistic_files)

    X = np.vstack((autistic_data, non_autistic_data))
    y = np.hstack((np.ones(autistic_data.shape[0]), np.zeros(non_autistic_data.shape[0])))
    return train_test_split(X, y, test_size=0.3, random_state=50)


def train_models():
    X_train, X_test, y_train, y_test = build_training_set()

    for model_path, model in MODEL_FILES.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"{model.__class__.__name__} accuracy: {acc:.2f}")
        joblib.dump(model, ROOT_DIR / model_path)


if __name__ == "__main__":
    train_models()



## 3. Streamlit Inference UI (`ui.py`)

Standalone application for uploading an audio file, extracting the averaged MFCC vector, and displaying the predicted label.


In [ ]:
import io
import tempfile
import warnings
from pathlib import Path

import joblib
import librosa
import numpy as np
import streamlit as st

warnings.filterwarnings("ignore", category=UserWarning)

ROOT_DIR = Path(__file__).resolve().parents[2]
MODEL_INFO = {
    "rf.pkl": "Random Forest (~90% accuracy)",
    "ann.pkl": "Artificial Neural Network (~72% accuracy)",
    "svm.pkl": "Support Vector Machine (~54% accuracy)",
    "nb.pkl": "Naive Bayes (~81% accuracy)",
}
N_MFCC = 40


def load_model(model_filename: str):
    model_path = ROOT_DIR / model_filename
    return joblib.load(model_path)


def extract_features(file_bytes: bytes) -> np.ndarray:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".m4a") as tmp_file:
        tmp_file.write(file_bytes)
        tmp_path = tmp_file.name

    try:
        y, sr = librosa.load(tmp_path, sr=None, mono=True)
        if y.size == 0:
            raise ValueError("Empty audio signal")

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
        if np.isnan(mfcc).any():
            raise ValueError("MFCC contains NaN values")

        mfcc_avg = np.mean(mfcc, axis=1)
        return mfcc_avg.reshape(1, -1)
    finally:
        Path(tmp_path).unlink(missing_ok=True)


st.title("Autism Detection")

selected_model_label = st.selectbox("Choose a model", list(MODEL_INFO.values()))
model_filename = next(key for key, value in MODEL_INFO.items() if value == selected_model_label)
model = load_model(model_filename)

uploaded_file = st.file_uploader("Upload an audio file (m4a/wav/mp3)", type=["m4a", "wav", "mp3"])

yes_style = '<h1 style="color:red;text-align:center;">Prediction: Autistic</h1>'
no_style = '<h1 style="color:green;text-align:center;">Prediction: Non Autistic</h1>'

if uploaded_file is not None:
    try:
        features = extract_features(uploaded_file.getvalue())
        prediction = model.predict(features)[0]
        if prediction == 1:
            st.markdown(yes_style, unsafe_allow_html=True)
        else:
            st.markdown(no_style, unsafe_allow_html=True)
    except Exception as exc:  # noqa: BLE001
        st.error(f"Could not process audio file: {exc}")



### Quick Usage

```bash
# 1. Extract features
python ../../mfcc_extract.py

# 2. Train models (saves rf.pkl, svm.pkl, nb.pkl, ann.pkl)
python ../../model.py

# 3. Launch the Streamlit UI
streamlit run ../../ui.py
```

